In [ ]:
#default_exp utils

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# utils

> Various utility functions used by the blurr package.

In [ ]:
#export
import sys, inspect
from enum import Enum

import torch
from transformers import *
from fastai.text.all import *

logging.set_verbosity_error()

In [ ]:
#hide_input
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.7.1
Using fastai 2.4
Using transformers 4.8.1


In [ ]:
#hide_input
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


In [ ]:
#export
def str_to_class(classname):
    "converts string representation to class"
    return getattr(sys.modules[__name__], classname)

In [ ]:
#export
class Singleton:
    def __init__(self,cls):
        self._cls, self._instance = cls, None

    def __call__(self, *args, **kwargs):
        if self._instance == None: self._instance = self._cls(*args, **kwargs)
        return self._instance

`Singleton` functions as python decorator.  Use this above any class to turn that class into a singleton (see [here](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html) for more info on the singleton pattern).

In [ ]:
@Singleton
class TestSingleton: pass

a = TestSingleton()
b = TestSingleton()
test_eq(a,b)

## BlurrUtil

In [ ]:
#export
@Singleton
class BlurrUtil():
    
    def __init__(self):
        # get hf classes (tokenizers, configs, models, etc...)
        transformer_classes = inspect.getmembers(sys.modules[__name__], 
                                                 lambda member: inspect.isclass(member)
                                                 and member.__module__.startswith('transformers.'))
        
        # build a df that we can query against to get various transformers objects/info
        self._df = pd.DataFrame(transformer_classes, columns=['class_name', 'class_location'])
        
        # add the module each class is included in
        self._df['module'] = self._df.class_location.apply(lambda v: v.__module__)
        
        # remove class_location (don't need it anymore)
        self._df.drop(labels=['class_location'], axis=1, inplace=True)
        
        # break up the module into separate cols
        module_parts_df = self._df.module.str.split(".", n = -1, expand = True) 
        for i in range(len(module_parts_df.columns)):
            self._df[f'module_part_{i}'] = module_parts_df[i]

        # using module part 1, break up the functional area and arch into separate cols
        module_part_3_df = self._df.module_part_3.str.split("_", n = 1, expand = True) 
        self._df[['functional_area', 'arch']] = module_part_3_df
        
        # transformers >=4.5.x does "auto" differently; so remove it and "utils" from "arch" column
        self._df = self._df[~self._df['arch'].isin(['auto', 'utils'])]
        

        # if functional area = modeling, pull out the task it is built for
        model_type_df = self._df[(self._df.functional_area == 'modeling')].class_name.str.split('For', n=1, expand=True)
        
        model_type_df[1] = np.where(model_type_df[1].notnull(), 
                                    'For' + model_type_df[1].astype(str), 
                                    model_type_df[1])
        
        self._df['model_task'] = model_type_df[1]
        self._df['model_task'] = self._df['model_task'].str.replace('For', '', n=1, case=True, regex=False)
        
        model_type_df = self._df[(self._df.functional_area == 'modeling')].class_name.str.split('With', n=1, expand=True)
        model_type_df[1] = np.where(model_type_df[1].notnull(), 
                                    'With' + model_type_df[1].astype(str), 
                                    self._df[(self._df.functional_area == 'modeling')].model_task)
        
        self._df['model_task'] = model_type_df[1]
        self._df['model_task'] = self._df['model_task'].str.replace('With', '', n=1, case=True, regex=False)
        
        # look at what we're going to remove (use to verify we're just getting rid of stuff we want too)
        # df[~df['hf_class_type'].isin(['modeling', 'configuration', 'tokenization'])]
        
        # only need these 3 functional areas for our querying purposes
        self._df = self._df[self._df['functional_area'].isin(['modeling', 'configuration', 'tokenization'])]
        
    def get_architectures(self): 
        """Used to get all the architectures supported by your `Transformers` install"""
        return sorted(self._df[(self._df.arch.notna()) & 
                        (self._df.arch != None)].arch.unique().tolist())
    
    def get_config(self, arch): 
        """Used the locate the name of the configuration class for a given architecture"""
        config = self._df[(self._df.functional_area == 'configuration') & 
                          (self._df.arch == arch)].class_name.values[0]
        
        return str_to_class(config)
    
    def get_tokenizers(self, arch): 
        """Used to get the available Hugging Face tokenizers for a given architecture. Note: There may be 
        multiple tokenizers and so this returns a list.
        """
        toks = sorted(self._df[(self._df.functional_area == 'tokenization') & 
                               (self._df.arch == arch)].class_name.values)
        
        return [str_to_class(tok_name) for tok_name in toks]
    
    def get_tasks(self, arch=None): 
        """Get the type of tasks for which there is a custom model for (*optional: by architecture*). 
        There are a number of customized models built for specific tasks like token classification, 
        question/answering, LM, etc....
        """
        query = ['model_task.notna()']
        if (arch): query.append(f'arch == "{arch}"')

        return sorted(self._df.query(' & '.join(query), engine='python').model_task.unique().tolist())
    
    def get_models(self, arch=None, task=None):
        """The transformer models available for use (optional: by architecture | task)"""
        query = ['functional_area == "modeling"']
        if (arch): query.append(f'arch == "{arch}"')
        if (task): query.append(f'model_task == "{task}"')

        models = sorted(self._df.query(' & '.join(query)).class_name.tolist())
        return [str_to_class(model_name) for model_name in models] 
    
    def get_classes_for_model(self, model_name_or_cls):
        """Get tokenizers, config, and model for a given model name / class"""
        model_name = model_name_or_cls if isinstance(model_name_or_cls, str) else model_name_or_cls.__name__

        meta = self._df[self._df.class_name == model_name]
        tokenizers = self.get_tokenizers(meta.arch.values[0])
        config = self.get_config(meta.arch.values[0])

        return (config, tokenizers, str_to_class(model_name))
    
    def get_model_architecture(self, model_name_or_enum):
        """Get the architecture for a given model name / enum"""
        model_name = model_name_or_enum if isinstance(model_name_or_enum, str) else model_name_or_enum.name
        return self._df[self._df.class_name == model_name].arch.values[0]
    
    def get_hf_objects(self, pretrained_model_name_or_path, task=None,
                       config=None, tokenizer_cls=None, model_cls=None, 
                       config_kwargs={}, tokenizer_kwargs={}, model_kwargs={}, cache_dir=None):
        """Returns the architecture (str), config (obj), tokenizer (obj), and model (obj) given at minimum a
        `pre-trained model name or path`. Specify a `task` to ensure the right "AutoModelFor<task>" is used to
        create the model.
        
        Optionally, you can pass a config (obj), tokenizer (class), and/or model (class) (along with any 
        related kwargs for each) to get as specific as you want w/r/t what Hugging Face objects are returned.
        """
        
        # config
        if (config is None):
            config = AutoConfig.from_pretrained(pretrained_model_name_or_path, cache_dir=cache_dir, **config_kwargs)
        
        # tokenizer (gpt2, roberta, bart (and maybe others) tokenizers require a prefix space)
        if (any(s in pretrained_model_name_or_path for s in ['gpt2', 'roberta', 'bart', 'longformer'])):
            tokenizer_kwargs = { **{'add_prefix_space': True}, **tokenizer_kwargs }

        if (tokenizer_cls is None):
            tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path, 
                                                      cache_dir=cache_dir, 
                                                      **tokenizer_kwargs)
        else:
            tokenizer = tokenizer_cls.from_pretrained(pretrained_model_name_or_path, 
                                                      cache_dir=cache_dir, 
                                                      **tokenizer_kwargs)
            
        # model
        if (model_cls is None and task is not None): 
                model_cls = self.get_models(task=task.name)[0]
        
        model = model_cls.from_pretrained(pretrained_model_name_or_path, 
                                          config=config, 
                                          cache_dir=cache_dir, 
                                          **model_kwargs)

        #arch
        arch = self.get_model_architecture(type(model).__name__)
        
        return (arch, config, tokenizer, model)

`BlurrUtil` is a `Singleton` (there exists only one instance, and the same instance is returned upon subsequent instantiation requests).  You can get at via the `BLURR` constant below.

In [ ]:
mh = BlurrUtil()
mh2 = BlurrUtil()
test_eq(mh, mh2)

In [ ]:
#hide
display_df(mh._df.head(20))

,class_name,module,module_part_0,module_part_1,module_part_2,module_part_3,functional_area,arch,model_task
3,AdaptiveEmbedding,transformers.models.transfo_xl.modeling_transfo_xl,transformers,models,transfo_xl,modeling_transfo_xl,modeling,transfo_xl,None
4,AlbertConfig,transformers.models.albert.configuration_albert,transformers,models,albert,configuration_albert,configuration,albert,NaN
5,AlbertForMaskedLM,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MaskedLM
6,AlbertForMultipleChoice,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MultipleChoice
7,AlbertForPreTraining,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,PreTraining
8,AlbertForQuestionAnswering,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,QuestionAnswering
9,AlbertForSequenceClassification,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,SequenceClassification
10,AlbertForTokenClassification,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,TokenClassification
11,AlbertModel,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,None
12,AlbertPreTrainedModel,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,None


In [ ]:
#hide
display_df(mh._df.head())

print(list(mh._df.model_task.unique()))
print('')
print(list(mh._df.functional_area.unique()))
print('')
print(list(mh._df.arch.unique()))
print('')
print(list(mh._df.module_part_3.unique()))

,class_name,module,module_part_0,module_part_1,module_part_2,module_part_3,functional_area,arch,model_task
3,AdaptiveEmbedding,transformers.models.transfo_xl.modeling_transfo_xl,transformers,models,transfo_xl,modeling_transfo_xl,modeling,transfo_xl,None
4,AlbertConfig,transformers.models.albert.configuration_albert,transformers,models,albert,configuration_albert,configuration,albert,NaN
5,AlbertForMaskedLM,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MaskedLM
6,AlbertForMultipleChoice,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MultipleChoice
7,AlbertForPreTraining,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,PreTraining


[None, nan, 'MaskedLM', 'MultipleChoice', 'PreTraining', 'QuestionAnswering', 'SequenceClassification', 'TokenClassification', 'CausalLM', 'ConditionalGeneration', 'NextSentencePrediction', 'ImageClassification', 'Teacher', 'QuestionAnsweringSimple', 'LMHeadModel', 'CTC', 'EntityClassification', 'EntityPairClassification', 'EntitySpanClassification', 'Classification', 'Generation', 'LMHead', 'merForCausalLM', 'merForMaskedLM', 'merForMultipleChoice', 'merForQuestionAnswering', 'merForSequenceClassification', 'merForTokenClassification', 'merLayer', 'merModel', 'merPreTrainedModel', 'RegionToPhraseAlignment', 'VisualReasoning']

['modeling', 'configuration', 'tokenization']

['transfo_xl', 'albert', 'albert_fast', 'bart', 'bart_fast', 'barthez', 'barthez_fast', 'bert', 'bert_generation', 'bert_japanese', 'bert_fast', 'bertweet', 'big_bird', 'bigbird_pegasus', 'big_bird_fast', 'blenderbot', 'blenderbot_small', 'byt5', 'clip', 'clip_fast', 'ctrl', 'camembert', 'camembert_fast', 'convbert'

### Provide global helper constant

Users of this library can simply use `BLURR` to access all the `BlurrUtil` capabilities without having to fetch an instance themselves.

In [ ]:
#export
BLURR = BlurrUtil()

Here's how you can get at the core Hugging Face objects you need to work with ...

... the ***task***

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_tasks)

<h4 id="BlurrUtil.get_tasks" class="doc_header"><code>BlurrUtil.get_tasks</code><a href="__main__.py#L78" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_tasks</code>(**`arch`**=*`None`*)

Get the type of tasks for which there is a custom model for (*optional: by architecture*). 
There are a number of customized models built for specific tasks like token classification, 
question/answering, LM, etc....

In [ ]:
print(BLURR.get_tasks())
print('')
print(BLURR.get_tasks('bart'))

['CTC', 'CausalLM', 'Classification', 'ConditionalGeneration', 'EntityClassification', 'EntityPairClassification', 'EntitySpanClassification', 'Generation', 'ImageClassification', 'LMHead', 'LMHeadModel', 'MaskedLM', 'MultipleChoice', 'NextSentencePrediction', 'PreTraining', 'QuestionAnswering', 'QuestionAnsweringSimple', 'RegionToPhraseAlignment', 'SequenceClassification', 'Teacher', 'TokenClassification', 'VisualReasoning', 'merForCausalLM', 'merForMaskedLM', 'merForMultipleChoice', 'merForQuestionAnswering', 'merForSequenceClassification', 'merForTokenClassification', 'merLayer', 'merModel', 'merPreTrainedModel']

['CausalLM', 'ConditionalGeneration', 'QuestionAnswering', 'SequenceClassification']


... the ***architecture***

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_architectures)

<h4 id="BlurrUtil.get_architectures" class="doc_header"><code>BlurrUtil.get_architectures</code><a href="__main__.py#L57" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_architectures</code>()

Used to get all the architectures supported by your `Transformers` install

In [ ]:
print(BLURR.get_architectures())

['albert', 'albert_fast', 'bart', 'bart_fast', 'barthez', 'barthez_fast', 'bert', 'bert_fast', 'bert_generation', 'bert_japanese', 'bertweet', 'big_bird', 'big_bird_fast', 'bigbird_pegasus', 'blenderbot', 'blenderbot_small', 'byt5', 'camembert', 'camembert_fast', 'clip', 'clip_fast', 'convbert', 'convbert_fast', 'cpm', 'ctrl', 'deberta', 'deberta_fast', 'deberta_v2', 'deit', 'detr', 'distilbert', 'distilbert_fast', 'dpr', 'dpr_fast', 'electra', 'electra_fast', 'encoder_decoder', 'flaubert', 'fsmt', 'funnel', 'funnel_fast', 'gpt2', 'gpt2_fast', 'gpt_neo', 'herbert', 'herbert_fast', 'hubert', 'ibert', 'layoutlm', 'layoutlm_fast', 'led', 'led_fast', 'longformer', 'longformer_fast', 'luke', 'lxmert', 'lxmert_fast', 'm2m_100', 'marian', 'mbart', 'mbart50', 'mbart50_fast', 'mbart_fast', 'megatron_bert', 'mmbt', 'mobilebert', 'mobilebert_fast', 'mpnet', 'mpnet_fast', 'mt5', 'openai', 'openai_fast', 'pegasus', 'pegasus_fast', 'phobert', 'prophetnet', 'rag', 'reformer', 'reformer_fast', 'retrib

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_model_architecture)

<h4 id="BlurrUtil.get_model_architecture" class="doc_header"><code>BlurrUtil.get_model_architecture</code><a href="__main__.py#L107" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_model_architecture</code>(**`model_name_or_enum`**)

Get the architecture for a given model name / enum

In [ ]:
print(BLURR.get_model_architecture('RobertaForSequenceClassification'))

roberta


... the ***config*** for that particular task and architecture

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_config)

<h4 id="BlurrUtil.get_config" class="doc_header"><code>BlurrUtil.get_config</code><a href="__main__.py#L62" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_config</code>(**`arch`**)

Used the locate the name of the configuration class for a given architecture

In [ ]:
print(BLURR.get_config('bert'))

<class 'transformers.models.bert.configuration_bert.BertConfig'>


... the available ***tokenizers*** for that architecture

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_tokenizers)

<h4 id="BlurrUtil.get_tokenizers" class="doc_header"><code>BlurrUtil.get_tokenizers</code><a href="__main__.py#L69" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_tokenizers</code>(**`arch`**)

Used to get the available Hugging Face tokenizers for a given architecture. Note: There may be 
multiple tokenizers and so this returns a list.

In [ ]:
print(BLURR.get_tokenizers('electra'))

[<class 'transformers.models.electra.tokenization_electra.ElectraTokenizer'>]


... and lastly the ***models*** (optionally for a given task and/or architecture)

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_models)

<h4 id="BlurrUtil.get_models" class="doc_header"><code>BlurrUtil.get_models</code><a href="__main__.py#L88" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_models</code>(**`arch`**=*`None`*, **`task`**=*`None`*)

The transformer models available for use (optional: by architecture | task)

In [ ]:
print(L(BLURR.get_models())[:5])

[<class 'transformers.models.transfo_xl.modeling_transfo_xl.AdaptiveEmbedding'>, <class 'transformers.models.albert.modeling_albert.AlbertForMaskedLM'>, <class 'transformers.models.albert.modeling_albert.AlbertForMultipleChoice'>, <class 'transformers.models.albert.modeling_albert.AlbertForPreTraining'>, <class 'transformers.models.albert.modeling_albert.AlbertForQuestionAnswering'>]


In [ ]:
print(BLURR.get_models(arch='bert')[:5])

[<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>, <class 'transformers.models.bert.modeling_bert.BertForMultipleChoice'>, <class 'transformers.models.bert.modeling_bert.BertForNextSentencePrediction'>, <class 'transformers.models.bert.modeling_bert.BertForPreTraining'>, <class 'transformers.models.bert.modeling_bert.BertForQuestionAnswering'>]


In [ ]:
print(BLURR.get_models(task='TokenClassification')[:5])

[<class 'transformers.models.albert.modeling_albert.AlbertForTokenClassification'>, <class 'transformers.models.bert.modeling_bert.BertForTokenClassification'>, <class 'transformers.models.big_bird.modeling_big_bird.BigBirdForTokenClassification'>, <class 'transformers.models.camembert.modeling_camembert.CamembertForTokenClassification'>, <class 'transformers.models.convbert.modeling_convbert.ConvBertForTokenClassification'>]


In [ ]:
print(BLURR.get_models(arch='bert', task='TokenClassification'))

[<class 'transformers.models.bert.modeling_bert.BertForTokenClassification'>]


Here we define some helpful enums to make it easier to get at the *architecture and task* you're looking for.

In [ ]:
#export
HF_ARCHITECTURES = Enum('HF_ARCHITECTURES', BLURR.get_architectures())

In [ ]:
print(L(HF_ARCHITECTURES)[:5])

[<HF_ARCHITECTURES.albert: 1>, <HF_ARCHITECTURES.albert_fast: 2>, <HF_ARCHITECTURES.bart: 3>, <HF_ARCHITECTURES.bart_fast: 4>, <HF_ARCHITECTURES.barthez: 5>]


In [ ]:
#export
HF_TASKS = Enum('HF_TASKS_ALL', BLURR.get_tasks())

In [ ]:
print('--- all tasks ---')
print(L(HF_TASKS))

--- all tasks ---
[<HF_TASKS_ALL.CTC: 1>, <HF_TASKS_ALL.CausalLM: 2>, <HF_TASKS_ALL.Classification: 3>, <HF_TASKS_ALL.ConditionalGeneration: 4>, <HF_TASKS_ALL.EntityClassification: 5>, <HF_TASKS_ALL.EntityPairClassification: 6>, <HF_TASKS_ALL.EntitySpanClassification: 7>, <HF_TASKS_ALL.Generation: 8>, <HF_TASKS_ALL.ImageClassification: 9>, <HF_TASKS_ALL.LMHead: 10>, <HF_TASKS_ALL.LMHeadModel: 11>, <HF_TASKS_ALL.MaskedLM: 12>, <HF_TASKS_ALL.MultipleChoice: 13>, <HF_TASKS_ALL.NextSentencePrediction: 14>, <HF_TASKS_ALL.PreTraining: 15>, <HF_TASKS_ALL.QuestionAnswering: 16>, <HF_TASKS_ALL.QuestionAnsweringSimple: 17>, <HF_TASKS_ALL.RegionToPhraseAlignment: 18>, <HF_TASKS_ALL.SequenceClassification: 19>, <HF_TASKS_ALL.Teacher: 20>, <HF_TASKS_ALL.TokenClassification: 21>, <HF_TASKS_ALL.VisualReasoning: 22>, <HF_TASKS_ALL.merForCausalLM: 23>, <HF_TASKS_ALL.merForMaskedLM: 24>, <HF_TASKS_ALL.merForMultipleChoice: 25>, <HF_TASKS_ALL.merForQuestionAnswering: 26>, <HF_TASKS_ALL.merForSequenceClas

In [ ]:
HF_TASKS.Classification

<HF_TASKS_ALL.Classification: 3>

`BLURR.get_classes_for_model` can be used to get the config, tokenizer, and model *classes* you want

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_classes_for_model)

<h4 id="BlurrUtil.get_classes_for_model" class="doc_header"><code>BlurrUtil.get_classes_for_model</code><a href="__main__.py#L97" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_classes_for_model</code>(**`model_name_or_cls`**)

Get tokenizers, config, and model for a given model name / class

In [ ]:
config, tokenizers, model = BLURR.get_classes_for_model('RobertaForSequenceClassification')

print(config)
print(tokenizers[0])
print(model)

<class 'transformers.models.roberta.configuration_roberta.RobertaConfig'>
<class 'transformers.models.roberta.tokenization_roberta.RobertaTokenizer'>
<class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'>


In [ ]:
config, tokenizers, model = BLURR.get_classes_for_model(DistilBertModel)

print(config)
print(tokenizers[0])
print(model)

<class 'transformers.models.distilbert.configuration_distilbert.DistilBertConfig'>
<class 'transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer'>
<class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'>


### Methods for loading pre-trained (configs, tokenizer, model) hugginface objects

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_hf_objects)

<h4 id="BlurrUtil.get_hf_objects" class="doc_header"><code>BlurrUtil.get_hf_objects</code><a href="__main__.py#L112" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_hf_objects</code>(**`pretrained_model_name_or_path`**, **`task`**=*`None`*, **`config`**=*`None`*, **`tokenizer_cls`**=*`None`*, **`model_cls`**=*`None`*, **`config_kwargs`**=*`{}`*, **`tokenizer_kwargs`**=*`{}`*, **`model_kwargs`**=*`{}`*, **`cache_dir`**=*`None`*)

Returns the architecture (str), config (obj), tokenizer (obj), and model (obj) given at minimum a
`pre-trained model name or path`. Specify a `task` to ensure the right "AutoModelFor<task>" is used to
create the model.

Optionally, you can pass a config (obj), tokenizer (class), and/or model (class) (along with any 
related kwargs for each) to get as specific as you want w/r/t what Hugging Face objects are returned.

In [ ]:
arch, config, tokenizer, model = BLURR.get_hf_objects("bert-base-cased-finetuned-mrpc",
                                                      model_cls=AutoModelForMaskedLM)

print(arch)
print(type(config))
print(type(tokenizer))
print(type(model))

bert
<class 'transformers.models.bert.configuration_bert.BertConfig'>
<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>


In [ ]:
arch, tokenizer, config, model = BLURR.get_hf_objects("fmikaelian/flaubert-base-uncased-squad",
                                                      model_cls=AutoModelForQuestionAnswering)

print(arch)
print(type(config))
print(type(tokenizer))
print(type(model))

flaubert
<class 'transformers.models.flaubert.tokenization_flaubert.FlaubertTokenizer'>
<class 'transformers.models.flaubert.configuration_flaubert.FlaubertConfig'>
<class 'transformers.models.flaubert.modeling_flaubert.FlaubertForQuestionAnsweringSimple'>


In [ ]:
arch, tokenizer, config, model = BLURR.get_hf_objects("bert-base-cased-finetuned-mrpc",
                                                      config=None,
                                                      tokenizer_cls=BertTokenizer, 
                                                      model_cls=BertForNextSentencePrediction)
print(arch)
print(type(config))
print(type(tokenizer)) 
print(type(model))

bert
<class 'transformers.models.bert.tokenization_bert.BertTokenizer'>
<class 'transformers.models.bert.configuration_bert.BertConfig'>
<class 'transformers.models.bert.modeling_bert.BertForNextSentencePrediction'>


## Summary

Use the `BLURR` object is optional, as you're free to build your Hugging Face objects manually which sometimes may be required given your use case.  Most of the time, however, `BLURR` is sufficent to get you everything you need in one-line with a pretrained model name or path.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted index.ipynb.
